https://github.com/Jaekyu-Sim/Unsupervised-Cross-Domain-Image-Generation-Network/blob/master/uscdg_model.ipynb

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import cv2
import os
import keras
import imageio

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
svhn_dir = '/content/drive/My Drive/lab2/svhn'
mnist_dir = '/content/drive/My Drive/lab2/mnist'
log_dir = '/content/drive/My Drive/lab2/log'
sample_save_path = '/content/drive/My Drive/lab2/sample'
model_save_path = '/content/drive/My Drive/lab2/model'
pretrained_model = '/content/drive/My Drive/lab2/model/svhn-11549'
test_model = '/content/drive/My Drive/lab2/model/dtn-0'

# tools



In [0]:

def load_svhn_data(test = False):
    if test:
      image_file = 'test_32x32.mat'
    else:
      image_file = 'extra_32x32.mat' 
    path = os.path.join(svhn_dir, image_file)
    data = loadmat(path)
    training_x = data['X']
    training_y = data['y'].reshape(-1)
    training_y[np.where(training_y==10)] = 0
    
    transposed_X_train = np.transpose(training_x, (3, 0, 1, 2))/127.5-1
    #print("transposed_X_train's shape : ",np.shape(transposed_X_train))
    return transposed_X_train,training_y

In [0]:
def create_list(data_length,batch_size):
    index = np.arange(0, data_length)
    np.random.shuffle(index)
    total_batch = int(data_length/batch_size)
    index =np.resize(index, total_batch*batch_size)

    return np.reshape(index,(-1,batch_size))

In [0]:
def load_mnist_data():
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
  reshaped_x = []
  for i in range(len(x_train)):
    reshaped_x.append(np.reshape(x_train[i], (28, 28)))
  resized_x = []
  for i in range(len(x_train)):
    resized_x.append(cv2.resize(reshaped_x[i], (32, 32)))
  output = np.reshape(resized_x, (len(x_train), 32, 32, 1))
  return output

In [0]:
batch_size = 100
def merge_images(sources, targets):
        _, h, w, _ = sources.shape
        row = int(np.sqrt(batch_size))
        merged = np.zeros([row*h, row*w*2, 3])
       
        for idx, (s, t) in enumerate(zip(sources, targets)):
            i = idx // row
            j = idx % row
            merged[i*h:(i+1)*h, (j*2)*h:(j*2+1)*h, :] = s
            merged[i*h:(i+1)*h, (j*2+1)*h:(j*2+2)*h, :] = t
        return merged

In [0]:
def data_normalize(data):
  res = map(lambda x: x/127.5-1, data)
  return list(res)

In [0]:
batch_size=100
def make_batch(data, batch_size):
    data_length = len(data)
    index = np.arange(0, data_length)
    np.random.shuffle(index)
    index = index[:batch_size]
    shuffled_data = [data[i] for i in index]
    
    return shuffled_data

In [0]:
def tf_reset():
    try:
        sess.close()
    except:
        pass
    tf.reset_default_graph()
    return tf.Session()

In [0]:
class model():
    def __init__(self, batch_size, d_lr,g_lr,epochs):
        self.batch_size = batch_size
        self.d_lr = d_lr
        self.g_lr = g_lr
        self.epochs = epochs
        self.img_width = 32
        self.img_height = 32
        self.img_channel = 3
        self.mnist_channel = 1
      
        self.src_domain_img = tf.placeholder(dtype = tf.float32, shape = [None, 32, 32, 3])
        self.trg_domain_img = tf.placeholder(dtype = tf.float32, shape = [None, 32, 32, 1])
        self.src_label = tf.placeholder(dtype = tf.int64, shape = [None])
        self.model()
        
    def feature_extractor(self, _input):
        if(np.shape(_input)[3] == 1):
            _input = tf.image.grayscale_to_rgb(_input)
        with tf.variable_scope("feature_extractor", reuse = tf.AUTO_REUSE):
            net = tf.layers.conv2d(inputs = _input, filters = 64, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)

            net = tf.layers.conv2d(inputs = net, filters = 128, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)

            net = tf.layers.conv2d(inputs = net, filters = 256, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)
            
            net = tf.layers.conv2d(inputs = net, filters = 128, kernel_size = [4, 4], padding = 'VALID', strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.tanh(net)
            
            
            return net
        
    def discriminator(self, _input):
        with tf.variable_scope("discriminator", reuse = tf.AUTO_REUSE):
            net = tf.layers.conv2d(inputs = _input, filters = 128, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)

            net = tf.layers.conv2d(inputs = net, filters = 256, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)

            net = tf.layers.conv2d(inputs = net, filters = 512, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)

            net = tf.layers.conv2d(inputs = net, filters = 1, kernel_size = [4, 4], padding = "VALID", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.flatten(net)
            return tf.nn.sigmoid(net), net
    
    def generator(self, _input):
        with tf.variable_scope("generator", reuse = tf.AUTO_REUSE):
            net = tf.layers.conv2d_transpose(inputs = _input, filters = 512, kernel_size = [4, 4], padding = "VALID", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)
            
            net = tf.layers.conv2d_transpose(inputs = net, filters = 256, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)
            
            net = tf.layers.conv2d_transpose(inputs = net, filters = 128, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.layers.batch_normalization(net, training=True, momentum=0.95)
            net = tf.nn.relu(net)
            
            net = tf.layers.conv2d_transpose(inputs = net, filters = self.mnist_channel, kernel_size = [3, 3], padding = "SAME", strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
            net = tf.nn.tanh(net)
            
            return net
    
    def model(self):
        #input -> self.src_domain_img, self.trg_domain_img
       
        self.src_fx = self.feature_extractor(self.src_domain_img)
        self.src_gfx = self.generator(self.src_fx)
        self.src_fgfx = self.feature_extractor(self.src_gfx)
        self.D1, self.D1_logits = self.discriminator(self.src_gfx)
        
        #feature_domain
        
       
        
        #target_domain 
        self.trg_fx = self.feature_extractor(self.trg_domain_img)
        self.trg_gfx = self.generator(self.trg_fx)
        self.D2, self.D2_logits = self.discriminator(self.trg_gfx)
        
        self.D3, self.D3_logits = self.discriminator(self.trg_domain_img)
        
        self.L_D1 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D1_logits, labels=tf.zeros_like(self.D1_logits)))
        self.L_D2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D2_logits, labels=tf.zeros_like(self.D2_logits)))
        self.L_D3 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D3_logits, labels=tf.ones_like(self.D3_logits)))
        
        self.Loss_D = self.L_D1 + self.L_D2 + self.L_D3
        
        self.L_G1 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D1_logits, labels = tf.ones_like(self.D1_logits)))
        self.L_G2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D2_logits, labels = tf.ones_like(self.D2_logits)))
        
        self.L_Gang = self.L_G1 + self.L_G2
        
        self.L_const = tf.reduce_mean(tf.square(self.src_fx - self.src_fgfx))
        self.L_tid = tf.reduce_mean(tf.square(self.trg_domain_img - self.trg_gfx))
        
        alpha = 15
        beta = 15
      
        self.Loss_G = self.L_Gang + beta*self.L_tid
        self.Loss_F = alpha*self.L_const
        
    def train(self):
        
        trainable_variables = tf.trainable_variables()
        #self.d_var = [var for var in trainable_variables if 'discriminator' in var.name]
        self.g_var = [var for var in trainable_variables if 'generator' in var.name]
        self.g_var.extend([var for var in trainable_variables if 'feature_extractor' in var.name])
        self.g_var.extend([var for var in trainable_variables if 'discriminator' in var.name])
        
        #model load
        
        #starter_learning_rate = self.g_lr
        #learning_rate_g = tf.compat.v1.train.exponential_decay(starter_learning_rate,global_step,500, 0.9, staircase=True)
        self.optimize_d = tf.train.AdamOptimizer(learning_rate = self.d_lr).minimize(self.Loss_D, var_list = self.g_var)#, global_step = global_step)
        self.optimize_g = tf.train.AdamOptimizer(learning_rate = self.g_lr).minimize(self.Loss_G, var_list = self.g_var)
        self.optimize_f = tf.train.AdamOptimizer(learning_rate = self.g_lr).minimize(self.Loss_F, var_list = self.g_var)
        
        #data load
        source_x,source_y = load_svhn_data()
        target_x = load_mnist_data()
       
        with tf.Session() as sess:
            
            wirter = tf.summary.FileWriter(model_save_path,sess.graph)
            
            sess.run(tf.global_variables_initializer())
            batch_size = self.batch_size
            num_of_data = len(source_x)
            total_batch = int(num_of_data / batch_size)
            
            #weight load
            restorer = tf.train.Saver(self.g_var)
            saver = tf.train.Saver()
            
            try:
                print("Existed Weight.ckpt load")
                restorer.restore(sess, pretrained_model)
            except:
                print("No Weight exist")
                print("Start Training with newly made Weight.ckpt")
            
            print("train start")
            loss_gen_list = []
            loss_dis_list = []
            loss_fea_list = []
            
            for epoch in range(self.epochs):
                
                source_list = create_list(len(source_x),self.batch_size)
                target_list = create_list(len(target_x),self.batch_size)
                print("epoch",epoch)
                print("---------------------------------------------------------")
                for iteration in range(total_batch):
                    #data load
                    global_step=epoch*total_batch+iteration

                    i = iteration% total_batch
                    batch_source_x= [source_x[m] for m in source_list[i]]
                    no_batch_source_x = data_normalize(batch_source_x)
                    #batch_source_x = color_img_normalize(batch_source_x)

                    j = iteration% int((len(target_x)/batch_size))
                    batch_target_x = [target_x[m] for m in target_list[j]]
                    no_batch_target_x = data_normalize(batch_target_x)
                    
                    #train start
                  
                    if global_step<=100:
                      repeated =2
                    else:
                      repeated =1
                    for i in range(repeated):
                      _, generator_loss = sess.run([self.optimize_g, self.Loss_G], feed_dict = {self.src_domain_img : no_batch_source_x, self.trg_domain_img : no_batch_target_x})
                     
                    _, discriminator_loss = sess.run([self.optimize_d, self.Loss_D], feed_dict = {self.src_domain_img : no_batch_source_x, self.trg_domain_img : no_batch_target_x})
                  

                    if global_step<100:
                      frequence = 50
                    else:
                      frequence = 60


                    if global_step % frequence ==0:
                      _, feature_extractor_loss = sess.run([self.optimize_f, self.Loss_F], feed_dict = {self.src_domain_img : no_batch_source_x})
                      print(iteration, "iteration  loss_f : ", feature_extractor_loss)

                    loss_dis_list.append(discriminator_loss)
                    loss_gen_list.append(generator_loss)
                    #loss_fea_list.append(feature_extractor_loss)


      
                    if(global_step%10== 0):
                      
                    
                      print(iteration, "iteration  loss_d : ", discriminator_loss)                
                      print(iteration, "iteration  loss_g : ", generator_loss)
                      

                      
                      
                    if(global_step%100== 0):
                      saver.save(sess, os.path.join(model_save_path, 'dtn'), global_step=global_step)
                      output = sess.run(self.src_gfx, feed_dict = {self.src_domain_img : batch_source_x}) 
                      
                  

                      for i in range(self.batch_size):
                        for row in range(self.img_width):
                            for col in range(self.img_height):
                                for channel in range(self.mnist_channel):
                                    output[i][row][col][channel] = int((output[i][row][col][channel]+1)*127.5)
                      output = np.reshape(output, (self.batch_size, self.img_width, self.img_height))
                      fig, ax = plt.subplots(1, self.batch_size, figsize=(self.batch_size, 2))
                      for i in range(self.batch_size):
                          ax[i].set_axis_off()
                          ax[i].imshow(batch_source_x[i])
                         
                      
                      fig2, ax2 = plt.subplots(1, self.batch_size, figsize=(self.batch_size, 2))
                      for i in range(self.batch_size):
                          ax2[i].set_axis_off()
                          ax2[i].imshow(output[i])
                      plt.show()

        
                plt.plot(loss_dis_list)
                plt.title("discriminator's loss")
                plt.show()
                  
                plt.plot(loss_gen_list)
                plt.title("generator's loss")
                plt.show()
                      
    def pretrain(self):
      trainable_variables = tf.trainable_variables()
      self.optimize_d = tf.train.AdamOptimizer(learning_rate = self.d_lr).minimize(self.Loss_D, var_list = self.g_var)#, global_step = global_step)
      self.optimize_g = tf.train.AdamOptimizer(learning_rate = self.g_lr).minimize(self.Loss_G, var_list = self.g_var)
      self.optimize_f = tf.train.AdamOptimizer(learning_rate = self.g_lr).minimize(self.Loss_F, var_list = self.g_var)

      net = self.feature_extractor(self.src_domain_img)
      net = tf.layers.conv2d(inputs = net, filters = 10, kernel_size = [1, 1], padding = 'VALID', strides = (2, 2), kernel_initializer=tf.contrib.layers.xavier_initializer())
      self.f_logits = tf.layers.flatten(net)
      
      
      self.pred = tf.argmax(self.f_logits, 1)
      self.correct_pred = tf.equal(self.pred, self.src_label)
      self.f_accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32)) 
      self.f_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.f_logits, labels=self.src_label))
     
      self.optimize_f = tf.train.AdamOptimizer(learning_rate = self.g_lr).minimize(self.f_loss, var_list = self.g_var)
        
      #data load
      source_x,source_y = load_svhn_data()
      source_x_test,source_y_test = load_svhn_data(test=True)
      
     
      with tf.Session() as sess:
          
          wirter = tf.summary.FileWriter(model_save_path,sess.graph)
          
          sess.run(tf.global_variables_initializer())
          batch_size = self.batch_size
          num_of_data = len(source_x)
          total_batch = int(num_of_data / batch_size)
          
          #weight load
          saver = tf.train.Saver(self.g_var)
          
          try:
              print("Existed Weight.ckpt load")
              saver = tf.train.Saver()
              saver.restore(sess, pretrained_model)
          except:
              print("No Weight exist")
              print("Start Training with newly made Weight.ckpt")            
          print("train start")
          
          for epoch in range(self.epochs):
              
              source_list = create_list(len(source_x),self.batch_size)
             
              print("epoch",epoch)
              print("---------------------------------------------------------")
     
      
              for iteration in range(total_batch):
                  #data load
                  i = iteration% total_batch
                  batch_source_x= [source_x[m] for m in source_list[i]]
                  batch_source_y= [source_y[m] for m in source_list[i]]
                  no_batch_source_x = data_normalize(batch_source_x)
                  
                  #train start

                  _, f_loss = sess.run([self.optimize_f, self.f_loss], feed_dict = {self.src_domain_img : no_batch_source_x, self.src_label : batch_source_y})
                  
                  if(iteration%10== 0):
                    print(iteration, "iteration  f_loss : ", f_loss)                
                    val_a, val_loss = sess.run([self.f_accuracy, self.f_loss], feed_dict = {self.src_domain_img : source_x_test, self.src_label : source_y_test})
                    print("val_acc : ", val_a,"val_loss:", val_loss)     
                  if val_a >0.9 or iteration%100==0:
                    print("val_acc : ", val_a,"val_loss:", val_loss)     
                    saver.save(sess, os.path.join(model_save_path, 'svhn'), global_step=epoch*total_batch+iteration)




    def test(self):
      svhn_images,_ = load_svhn_data(test = True)
      with tf.Session() as sess:
        # load trained parameters
        print ('loading test .')
        saver = tf.train.Saver()
        saver.restore(sess, test_model)

        print ('start sampling..!')
        for i in range(3):
            # train model for source domain S
            batch_images = svhn_images[i*self.batch_size:(i+1)*self.batch_size]
            
            feed_dict = {self.src_domain_img: batch_images}
            sampled_batch_images = sess.run(self.src_gfx, feed_dict)

            # merge and save source images and sampled target images
            merged = merge_images(batch_images, sampled_batch_images)
            path = os.path.join(sample_save_path, 'sample-%d-to-%d.png' %(i*batch_size, (i+1)*batch_size))
            imageio.imwrite(path, merged)
            print ('saved %s' %path)

# pretrain

In [0]:
tf_reset()

In [0]:
model = model(100,0.0003,0.0003,30)

In [0]:
model.pretrain()

Existed Weight.ckpt load
No Weight exist
Start Training with newly made Weight.ckpt
train start
epoch 0
---------------------------------------------------------
0 iteration  f_loss :  2.659245
val_acc :  0.118008606 val_loss: 2.5372415
val_acc :  0.118008606 val_loss: 2.5372415
10 iteration  f_loss :  2.0849066
val_acc :  0.24927013 val_loss: 2.175567
20 iteration  f_loss :  2.0323586
val_acc :  0.32709742 val_loss: 2.0054228
30 iteration  f_loss :  1.6973922
val_acc :  0.3847188 val_loss: 1.8538008
40 iteration  f_loss :  1.4610485
val_acc :  0.42762753 val_loss: 1.7212415
50 iteration  f_loss :  1.1428968
val_acc :  0.52408576 val_loss: 1.4915321
60 iteration  f_loss :  1.1114193
val_acc :  0.5459819 val_loss: 1.4074626
70 iteration  f_loss :  1.0013814
val_acc :  0.61493546 val_loss: 1.2364906
80 iteration  f_loss :  0.7770984
val_acc :  0.63752306 val_loss: 1.1529244
90 iteration  f_loss :  0.761335
val_acc :  0.6695221 val_loss: 1.0731881
100 iteration  f_loss :  0.8237262
val_ac

KeyboardInterrupt: ignored

# train()

In [0]:
tf_reset()

In [0]:
model = model(100,0.00001,0.00001,20)#1,2,20

In [0]:
model.train()

# test

In [0]:
test_model = '/content/drive/My Drive/lab2/model/dtn-2600'

In [0]:
tf_reset()

In [0]:
model = model(100,0.00001,0.00001,20)#1,2,20

In [0]:
model.test()

loading test .
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/lab2/model/dtn-2600


INFO:tensorflow:Restoring parameters from /content/drive/My Drive/lab2/model/dtn-2600


start sampling..!


saved /content/drive/My Drive/lab2/sample/sample-0-to-100.png


saved /content/drive/My Drive/lab2/sample/sample-100-to-200.png


saved /content/drive/My Drive/lab2/sample/sample-200-to-300.png
